In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

justinjyoo_ai4all_path = kagglehub.dataset_download('justinjyoo/ai4all')

print('Data source import complete.')


In [ ]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Get Dataset
link = "/kaggle/input/ai4all/mxmh_survey_results (1).csv"
df = pd.read_csv(link)
df.head()

,Timestamp,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,Music effects,Permissions
0,8/27/2022 19:29:02,18.0,Spotify,3.0,Yes,Yes,Yes,Latin,Yes,Yes,...,Sometimes,Very frequently,Never,Sometimes,3.0,0.0,1.0,0.0,NaN,I understand.
1,8/27/2022 19:57:31,63.0,Pandora,1.5,Yes,No,No,Rock,Yes,No,...,Sometimes,Rarely,Very frequently,Rarely,7.0,2.0,2.0,1.0,NaN,I understand.
2,8/27/2022 21:28:18,18.0,Spotify,4.0,No,No,No,Video game music,No,Yes,...,Never,Rarely,Rarely,Very frequently,7.0,7.0,10.0,2.0,No effect,I understand.
3,8/27/2022 21:40:40,61.0,YouTube Music,2.5,Yes,No,Yes,Jazz,Yes,Yes,...,Sometimes,Never,Never,Never,9.0,7.0,3.0,3.0,Improve,I understand.
4,8/27/2022 21:54:47,18.0,Spotify,4.0,Yes,No,No,R&B,Yes,No,...,Very frequently,Very frequently,Never,Rarely,7.0,2.0,5.0,9.0,Improve,I understand.


In [ ]:
# Remove columns not needed for analysis
# Includes Timestamp, Primary Streaming Service, BPM, Permissions
df.drop(columns=["Timestamp","Primary streaming service","BPM",
                "Permissions"], inplace = True, errors = 'ignore')

In [ ]:
# Note: Data Shape is (719, 29)
print(df.shape)

(736, 29)


In [ ]:
# Drop all rows with missing values (Total of 17 Rows, Now at 719 Rows)
df.dropna(axis = 0, inplace = True)

# Note: no more missing values

In [ ]:
# Set Target Variable and Features
# We know from my first code draft that we have no outliers since we are not removing ages
y = df['Music effects']
X = df.drop("Music effects", axis=1)

In [ ]:
# Split Data
from sklearn.model_selection import train_test_split

# First split: 100% → 80% temp + 20% test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Second split: 80% temp → 60% train + 20% validation
# To get 20% validation from 80% temp: 20/80 = 0.25
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=0)

In [ ]:
# Setup a model to test Ordinal Encoding vs. One-Hot Encoding
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_val, y_train, y_val):
    model = RandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    return mean_absolute_error(y_val, predictions)

In [ ]:
# Test Ordinal Encoding
from sklearn.preprocessing import OrdinalEncoder

# Get list of categorical variables for X
s = (X_train.dtypes == 'object')
X_object_cols = list(s[s].index)

# Make copy to avoid changing original data
label_X_train = X_train.copy()
label_X_valid = X_val.copy()
label_y_train = y_train.copy()
label_y_valid = y_val.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[X_object_cols] = ordinal_encoder.fit_transform(X_train[X_object_cols])
label_X_valid[X_object_cols] = ordinal_encoder.transform(X_val[X_object_cols])

# Do the same for target variable
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_y_train = label_encoder.fit_transform(y_train)
label_y_valid = label_encoder.transform(y_val)

print(score_dataset(label_X_train, label_X_valid, label_y_train, label_y_valid))

# Interpret Scale
# Yes / No:
    # 0 - Yes
    # 1 - No
# Likert Scale
    # 0 - Never
    # 1 - Rarely
    # 2 - Sometimes
    # 3 - Very Frequently
# Improve, Worsen, No effect
    # 0 - Improve
    # 1 - No effect
    # 2 - Worsen

0.3055555555555556


In [ ]:
# Test One-Hot Encodidng
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[X_object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_val[X_object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_val.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(X_object_cols, axis=1)
num_X_valid = X_val.drop(X_object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):")
print(score_dataset(OH_X_train, OH_X_valid, label_y_train, label_y_valid))

# Interpret
# Although the number is extremely close, Ordinal Encoding has a smaller MAE

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


MAE from Approach 3 (One-Hot Encoding):
0.3194444444444444


In [ ]:
# Find Optimal Num of Trees

# Function that calculates MAE based on n_estimators and data
def get_mae(n_estimators, label_X_train, label_X_valid, label_y_train, label_y_valid):
    final_model = RandomForestClassifier(n_estimators=n_estimators, random_state=0)
    final_model.fit(label_X_train, label_y_train)

    predictions = final_model.predict(label_X_valid)

    mae = mean_absolute_error(label_y_valid, predictions)
    return(mae)

candidate_n_estimators = [10, 50, 100, 500]

scores = {num_n: get_mae(num_n, label_X_train, label_X_valid,
                         label_y_train, label_y_valid)
          for num_n in candidate_n_estimators}

# minimum score = min MAE + corresponding n_estimators = 50
best_size = min(scores, key=scores.get)

for n, mae in scores.items():
    print(f"n_estimators: {n}\tMAE: {mae:.3f}")

n_estimators: 10	MAE: 0.333
n_estimators: 50	MAE: 0.326
n_estimators: 100	MAE: 0.306
n_estimators: 500	MAE: 0.306


In [ ]:
# Validate
# Use Ordinal Encoding Split Data

final_model = RandomForestClassifier(n_estimators=100, random_state=0)
final_model.fit(label_X_train, label_y_train)

validationMAE = final_model.predict(label_X_valid)
print(mean_absolute_error(label_y_valid, predictions))

0.3055555555555556


In [ ]:
# Test final model on unseen test data
# Apply same encoding to test set first
test_X_encoded = X_test.copy()
test_X_encoded[X_object_cols] = ordinal_encoder.transform(X_test[X_object_cols])
test_y_encoded = label_encoder.transform(y_test)

# Make predictions on test set
test_predictions = final_model.predict(test_X_encoded)
print("Test Set MAE:", mean_absolute_error(test_y_encoded, test_predictions))

Test Set MAE: 0.2361111111111111
